In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv("/Users/pranaymishra/Desktop/sih1429/ommas_main/static/data/monitor/sqm_monitors_jh.csv")




In [24]:
df

,MONITOR_NAME,ADMIN_IM_YEAR,ADMIN_IM_MONTH1,STATE_NAME,DISTRICT_NAME,COMP_S,COMP_SRI,COMP_U,C_GRADE_COUNT_T,ONGOING_S,...,ONGOING_U,P_GRADE_COUNT_T,MAINT_S,MAINT_SRI,MAINT_U,M_GRADE_COUNT_T,LSB_S,LSB_SRI,LSB_U,L_GRADE_COUNT_T
0,Aftab Alam,2023,January,Jharkhand,Dhanbad,0,0,0,0,11,...,0,11,0,0,0,0,0,0,0,0
1,Aftab Alam,2023,March,Jharkhand,Simdega,2,0,0,2,1,...,0,1,6,0,0,6,2,0,0,2
2,Aftab Alam,2023,May,Jharkhand,Simdega,0,0,0,0,0,...,0,0,5,0,0,5,0,0,0,0
3,Aftab Alam,2023,June,Jharkhand,Sahibganj,0,0,0,0,4,...,0,4,0,0,0,0,0,0,0,0
4,Aftab Alam,2023,October,Jharkhand,Dumka,5,0,0,5,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Yogendra Kumar Pandit,2023,August,Jharkhand,Chatra,2,0,0,2,6,...,0,6,5,0,0,5,1,0,0,1
201,Yogendra Kumar Pandit,2023,August,Jharkhand,Koderma,0,0,0,0,2,...,0,2,0,0,0,0,0,0,0,0
202,Yogendra Kumar Pandit,2023,September,Jharkhand,Hazaribagh,1,0,0,1,3,...,0,3,0,0,0,0,0,0,0,0
203,Yogendra Kumar Pandit,2023,October,Jharkhand,Deoghar,0,0,0,0,0,...,0,0,3,0,0,3,7,0,0,7


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming 'target_variable' is your target variable
X = df.drop(['MONITOR_NAME', 'COMP_S'], axis=1)
y = df['COMP_S']

# Convert categorical variables to numerical format using one-hot encoding
X_encoded = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Choose and train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the testing set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7560975609756098


In [27]:
# Assuming 'model' is your trained machine learning model

# New data (replace this with the actual data you want to predict on)
new_data = pd.DataFrame({
    'ADMIN_IM_YEAR': [2023],
    'ADMIN_IM_MONTH1': ['January'],
    'STATE_NAME': ['Jharkhand'],
    'DISTRICT_NAME': ['Dhanbad'],
    'COMP_S': [0],
    'COMP_SRI': [0],
    'COMP_U': [0],
    # ... Include other features based on your trained model
})

# Preprocess the new data (similar to how you preprocessed your training data)
new_data_encoded = pd.get_dummies(new_data)

# Make predictions
predictions = model.predict(new_data_encoded)

# Print or use the predictions as needed
print(predictions)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- COMP_S
Feature names seen at fit time, yet now missing:
- ADMIN_IM_MONTH1_April
- ADMIN_IM_MONTH1_August
- ADMIN_IM_MONTH1_December
- ADMIN_IM_MONTH1_February
- ADMIN_IM_MONTH1_July
- ...
